# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,tolanaro,-25.0319,46.9987,291.19,94,20,2.06,MG,1687894608
1,1,tiksi,71.6872,128.8694,282.35,64,99,4.15,RU,1687894608
2,2,waitangi,-43.9535,-176.5597,284.72,94,100,4.92,NZ,1687894608
3,3,west island,-12.1568,96.8225,299.14,78,40,6.17,CC,1687894608
4,4,ephrata,40.1798,-76.1788,299.96,83,40,3.60,US,1687894609


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,    
    size = "Humidity",
    scale = 1,
    color = "City"
)

# Display the map
map_plot_1



:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
best_weather = city_data_df.loc[(city_data_df['Humidity']<80) & (city_data_df['Cloudiness']<50)]

# Drop any rows with null values
best_weather_clean = best_weather.dropna()


# Display sample data
best_weather

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
3,3,west island,-12.1568,96.8225,299.14,78,40,6.17,CC,1687894608
6,6,poxoreo,-15.8372,-54.3892,303.77,27,0,2.86,BR,1687894609
7,7,minas de marcona,-15.2119,-75.1103,290.41,58,0,5.49,PE,1687894610
16,16,marsh harbour,26.5412,-77.0636,302.33,72,6,4.12,BS,1687894517
17,17,huarmey,-10.0681,-78.1522,296.54,68,11,5.72,PE,1687894613
...,...,...,...,...,...,...,...,...,...,...
545,545,ondjiva,-17.0667,15.7333,284.74,48,6,2.40,AO,1687894779
548,548,ladybrand,-29.1945,27.4574,282.77,44,30,4.90,ZA,1687894780
549,549,natal,-5.7950,-35.2094,302.27,61,20,6.69,BR,1687894605
558,558,umm kaddadah,13.6017,26.6876,305.79,35,10,3.99,SD,1687894784


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[['City','Country','Lat','Lng','Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
header_list = ['City','Country','Lat','Lng','Humidity','Hotel Name']
hotel_df = hotel_df.reindex(columns = header_list)
# Display sample data
hotel_df 

,City,Country,Lat,Lng,Humidity,Hotel Name
0,tolanaro,MG,-25.0319,46.9987,94,NaN
1,tiksi,RU,71.6872,128.8694,64,NaN
2,waitangi,NZ,-43.9535,-176.5597,94,NaN
3,west island,CC,-12.1568,96.8225,78,NaN
4,ephrata,US,40.1798,-76.1788,83,NaN
...,...,...,...,...,...,...
556,sinabang,ID,2.4803,96.3801,83,NaN
557,douentza,ML,14.9951,-2.9517,76,NaN
558,umm kaddadah,SD,13.6017,26.6876,35,NaN
559,bougouni,ML,11.4170,-7.4790,77,NaN


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:

# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
}
# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search


tolanaro - nearest hotel: Hôtel Mahavokey
tiksi - nearest hotel: Арктика
waitangi - nearest hotel: Hotel Chathams
west island - nearest hotel: Cocos Village Bungalows
ephrata - nearest hotel: 1777 Americana Inn
rengo - nearest hotel: Hotel Santa Cecilia
poxoreo - nearest hotel: No hotel found
minas de marcona - nearest hotel: No hotel found
svetlyy - nearest hotel: Гостиница Спортивная
iqaluit - nearest hotel: Frobisher Inn
adamstown - nearest hotel: No hotel found
alausi - nearest hotel: LA CASITA Guest House
college - nearest hotel: BEST WESTERN PLUS Pioneer Park Inn
labrador city - nearest hotel: Carol Inn
kataba - nearest hotel: No hotel found
new plymouth - nearest hotel: Carrington Motel
marsh harbour - nearest hotel: Lofty Fig Villas
huarmey - nearest hotel: Hostal Santa Rosa
puerto colombia - nearest hotel: Hotel Puerto Colombia
bengkulu - nearest hotel: Dena Hotel
bethel - nearest hotel: Hampton Inn Danbury
colonia - nearest hotel: Wasserturm Hotel Cologne
avarua - nearest hot

,City,Country,Lat,Lng,Humidity,Hotel Name
0,tolanaro,MG,-25.0319,46.9987,94,Hôtel Mahavokey
1,tiksi,RU,71.6872,128.8694,64,Арктика
2,waitangi,NZ,-43.9535,-176.5597,94,Hotel Chathams
3,west island,CC,-12.1568,96.8225,78,Cocos Village Bungalows
4,ephrata,US,40.1798,-76.1788,83,1777 Americana Inn
...,...,...,...,...,...,...
556,sinabang,ID,2.4803,96.3801,83,No hotel found
557,douentza,ML,14.9951,-2.9517,76,No hotel found
558,umm kaddadah,SD,13.6017,26.6876,35,No hotel found
559,bougouni,ML,11.4170,-7.4790,77,Hotel Piemont


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [9]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,    
    size = "Humidity",
    scale = 1,
    color = "City",
    hover_cols = "all"
)

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,index,Country,Hotel Name)